In [1]:
from face_alignment import FaceAlignment, LandmarksType

from giskard_vision.landmark_detection.dataloaders.loaders import DataLoaderFFHQ, DataLoader300W
from giskard_vision.landmark_detection.dataloaders.wrappers import (
    CroppedDataLoader,
    ResizedDataLoader,
    ColoredDataLoader,
    BlurredDataLoader,
    FilteredDataLoader,
    HeadPoseDataLoader,
    EthnicityDataLoader,
    CachedDataLoader,
)

from giskard_vision.landmark_detection.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from giskard_vision.landmark_detection.tests.performance import NMEMean
from giskard_vision.landmark_detection.marks.facial_parts import FacialParts
from giskard_vision.landmark_detection.tests.report import Report

In [2]:
dl_ref = DataLoader300W(dir_path="../datasets/300W/sample")

In [3]:
# cropping
dl_cropped_left = CroppedDataLoader(dl_ref, part=FacialParts.LEFT_HALF.value)
dl_cropped_upper = CroppedDataLoader(dl_ref, part=FacialParts.UPPER_HALF.value)

# resizing
dl_resized = ResizedDataLoader(dl_ref, scales=0.5)

# coloring
dl_colored = ColoredDataLoader(dl_ref)

# blurring
dl_blurred = BlurredDataLoader(dl_ref)


# head pose filtering
def positive_roll(elt):
    return elt[2]["headPose"]["roll"] > 0


def negative_roll(elt):
    return elt[2]["headPose"]["roll"] < 0


cached_dl = CachedDataLoader(HeadPoseDataLoader(dl_ref), cache_size=None, cache_img=False, cache_marks=False)
dl_positive_roll = FilteredDataLoader(cached_dl, positive_roll)
dl_negative_roll = FilteredDataLoader(cached_dl, negative_roll)


# ethnicity filtering
def white_ethnicity(elt):
    return elt[2]["ethnicity"] == "white"


def latino_ethnicity(elt):
    return elt[2]["ethnicity"] == "latino hispanic"


cached_dl = CachedDataLoader(
    EthnicityDataLoader(dl_ref, ethnicity_map={"indian": "asian"}), cache_size=None, cache_img=False, cache_marks=False
)
dl_white = FilteredDataLoader(cached_dl, white_ethnicity)
dl_latino = FilteredDataLoader(cached_dl, latino_ethnicity)

dataloaders_list = [
    dl_cropped_left,
    dl_cropped_upper,
    dl_resized,
    dl_colored,
    dl_blurred,
    dl_positive_roll,
    dl_negative_roll,
    dl_white,
    dl_latino,
]

2024-01-17 12:01:44.698306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
models_list = [
    FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False, face_detector="blazeface")),
    OpenCVWrapper(),
]

models_list = [models_list[1]]

loading data from : lbfmodel.yaml


In [5]:
report = Report(models_list, dataloaders_list, dataloader_ref=dl_ref)

OpenCVWrapper: Face not detected in processed image of batch 1 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 5 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 1 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 2 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 3 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 5 and index 0.


In [6]:
report.to_dataframe()

,model,facial_part,dataloader,prediction_time,prediction_fail_rate,test,metric,metric_value,threshold,passed
0,OpenCV,left half,300W cropped on left half,0.929891,0.564706,TestDiff,NME_mean,-0.644057,-0.1,True
1,OpenCV,upper half,300W cropped on upper half,0.943296,0.682353,TestDiff,NME_mean,0.040216,-0.1,False
2,OpenCV,entire face,300W resizing with ratios: 0.5,0.995143,0.000000,TestDiff,NME_mean,-0.079876,-0.1,False
3,OpenCV,entire face,300W altered with color mode 7,1.316452,0.000000,TestDiff,NME_mean,0.001347,-0.1,False
4,OpenCV,entire face,300W blurred,1.483105,0.000000,TestDiff,NME_mean,-0.103017,-0.1,True
5,OpenCV,entire face,(Cached (300W) with head-pose) filtered using ...,0.974038,0.000000,TestDiff,NME_mean,0.077927,-0.1,False
6,OpenCV,entire face,(Cached (300W) with head-pose) filtered using ...,2.076661,0.000000,TestDiff,NME_mean,-0.019482,-0.1,False
7,OpenCV,entire face,(Cached (300W) with ethnicity) filtered using ...,2.389328,0.000000,TestDiff,NME_mean,0.168421,-0.1,False
8,OpenCV,entire face,(Cached (300W) with ethnicity) filtered using ...,1.945159,0.000000,TestDiff,NME_mean,-0.784538,-0.1,True


In [7]:
report.adjust_thresholds({0: -0.9})
report.to_dataframe()

,model,facial_part,dataloader,prediction_time,prediction_fail_rate,test,metric,metric_value,threshold,passed
0,OpenCV,left half,300W cropped on left half,0.929891,0.564706,TestDiff,NME_mean,-0.644057,-0.9,False
1,OpenCV,upper half,300W cropped on upper half,0.943296,0.682353,TestDiff,NME_mean,0.040216,-0.1,False
2,OpenCV,entire face,300W resizing with ratios: 0.5,0.995143,0.000000,TestDiff,NME_mean,-0.079876,-0.1,False
3,OpenCV,entire face,300W altered with color mode 7,1.316452,0.000000,TestDiff,NME_mean,0.001347,-0.1,False
4,OpenCV,entire face,300W blurred,1.483105,0.000000,TestDiff,NME_mean,-0.103017,-0.1,True
5,OpenCV,entire face,(Cached (300W) with head-pose) filtered using ...,0.974038,0.000000,TestDiff,NME_mean,0.077927,-0.1,False
6,OpenCV,entire face,(Cached (300W) with head-pose) filtered using ...,2.076661,0.000000,TestDiff,NME_mean,-0.019482,-0.1,False
7,OpenCV,entire face,(Cached (300W) with ethnicity) filtered using ...,2.389328,0.000000,TestDiff,NME_mean,0.168421,-0.1,False
8,OpenCV,entire face,(Cached (300W) with ethnicity) filtered using ...,1.945159,0.000000,TestDiff,NME_mean,-0.784538,-0.1,True


In [8]:
import tempfile

with tempfile.NamedTemporaryFile() as f:
    report.to_json(filename=f.name)